Autor: Aleksandr Dmitriev (240259/xdmitr01)

In [1]:
import pandas as pd
from scipy.stats import chi2_contingency, ttest_ind, normaltest, mannwhitneyu

## Úkol 2: Test hypotézy

Naším cílem je ověřit dvě hypotézy na základě poskytnutých dat.

## Načtení dat
Načteme soubory `accidents.pkl.gz` a `vehicles.pkl.gz` pro analýzu.

In [2]:
accidents = pd.read_pickle("accidents.pkl.gz")
vehicles = pd.read_pickle("vehicles.pkl.gz")

## Prozkoumání dat
Zkontrolujeme základní informace o datasetu a prozkoumáme klíčové sloupce.

In [3]:
print("Accidents dataset info:")
print(accidents.info())
print("\nVehicles dataset info:")
print(vehicles.info())

Accidents dataset info:
<class 'pandas.core.frame.DataFrame'>
Index: 164646 entries, 0 to 69700
Data columns (total 40 columns):
 #   Column   Non-Null Count   Dtype         
---  ------   --------------   -----         
 0   p1       164646 non-null  int64         
 1   p2a      164646 non-null  object        
 2   p2b      164646 non-null  int64         
 3   p4a      164646 non-null  int64         
 4   p4b      164646 non-null  int64         
 5   p4c      164646 non-null  int64         
 6   p5a      164646 non-null  int64         
 7   p6       164646 non-null  int64         
 8   p7       164646 non-null  int64         
 9   p8       164646 non-null  int64         
 10  p8a      163580 non-null  float64       
 11  p9       164646 non-null  int64         
 12  p10      164646 non-null  int64         
 13  p11      164646 non-null  int64         
 14  p11a     163580 non-null  float64       
 15  p12      164646 non-null  int64         
 16  p13a     164646 non-null  int64       

## Hypotéza 1:

<em>Na silnicích první třídy byly nehody s následky na zdraví se stejnou pravděpodobností jako na dálnicích.</em>

K ověření použijeme 𝜒² test s tím, že také určíme, jestli nehody na silnicích 1. třídy
vedly častěji či méně často k nehodě s následky na zdraví (sloupec p9) než na dálnicích
(p36). V tom nám pomůže “expected” výstup 𝜒2 testu.

Přípravíme data pro Hypotézu 1. Ve sloupci p36 je uveden druh pozemní komunikace,
ve sloupci p9 je charakter nehody.

In [4]:
first_class = accidents[accidents['p36'] == 1]  # Silnice první třídy
d1_health = first_class['p9'].value_counts()
highways = accidents[accidents['p36'] == 0]  # Dálnice
d0_health = highways['p9'].value_counts()

Vytvoříme kontingenční tabulku pro test:

In [5]:
contingency_table = pd.DataFrame({
    "1. třída": d1_health,
    "Dálnice": d0_health
}).fillna(0)

Provedeme 𝜒² test:

In [6]:
chi2, p, dof, expected = chi2_contingency(contingency_table)

print("\nKontingenční tabulka:")
print(contingency_table)
print("\nVýsledky 𝜒² testu:")
print(f"𝜒²: {chi2}, p-hodnota: {p}, stupně volnosti: {dof}\n")
print("Očekávané hodnoty:")
print(expected)


Kontingenční tabulka:
    1. třída  Dálnice
p9                   
2      14773     6674
1       7059     1247

Výsledky 𝜒² testu:
𝜒²: 794.1532980454858, p-hodnota: 1.0075002730159729e-174, stupně volnosti: 1

Očekávané hodnoty:
[[15737.26696468  5709.73303532]
 [ 6094.73303532  2211.26696468]]


## Interpretace výsledků první hypotézy

Extrémně nízká hodnota **p** znamená, že rozdíl mezi pozorovanými a očekávanými hodnotami je statisticky velmi významný. Na hladině významnosti 0.05 **zamítáme** nulovou hypotézu, že rozdělení je stejné. Ve skutečnosti:
- Na silnicích první třídy došlo častěji k nehodám s následky na zdraví, než by se očekávalo.
- Na dálnicích bylo méně nehod s následky na zdraví, než by odpovídalo očekávání.

Očekávané hodnoty představují, jaké počty by měly být pozorovány, pokud by nehody s následky byly rozděleny rovnoměrně mezi silnice první třídy a dálnice. Tento výsledek naznačuje, že nehody s následky na zdraví mají odlišné pravděpodobnosti v závislosti na typu silnice.

## Hypotéza 2:
<em>Škoda při nehodách trolejbusů je nižší než při nehodách autobusů a tato odchylka je statisticky významná.</em>

Příprava dat pro Hypotézu 2. Ve sloupci p44 je uveden druh vozidla, 
ve sloupci p53 je uvedena škoda na vozidle ve stokorunách.

In [7]:
trolleybus_damage = vehicles[vehicles['p44'] == 11]['p53']  # Trolejbusy
bus_damage = vehicles[vehicles['p44'] == 8]['p53']  # Autobusy

Zkontrolujeme distribuce dat:

In [8]:
_, p_trolleybus = normaltest(trolleybus_damage)
_, p_bus = normaltest(bus_damage)

Pokud jsou data normálně rozložená, použijeme t-test, jinak použijeme Mann-Whitney U test

In [9]:
if p_trolleybus > 0.05 and p_bus > 0.05:
    stat, p_value = ttest_ind(trolleybus_damage, bus_damage, equal_var=False)
    test_used = "t-test"
else:
    stat, p_value = mannwhitneyu(trolleybus_damage, bus_damage)
    test_used = "Mann-Whitney U test"

print(f"Použitý test: {test_used}")
print(f"Mediány: Trolejbus - {trolleybus_damage.median()}, Autobus - {bus_damage.median()}")
print(f"Statistika testu: {stat}, p-hodnota: {p_value}")

Použitý test: Mann-Whitney U test
Mediány: Trolejbus - 50.0, Autobus - 200.0
Statistika testu: 469902.5, p-hodnota: 3.5404196579912076e-13


## Interpretace výsledků druhé hypotézy

Mediánová škoda při nehodách autobusů je tedy čtyřikrát vyšší než u trolejbusů. Extrémně nízká hodnota **p-value** (daleko pod hladinou významnosti 0.05) naznačuje, že rozdíl mezi škodami u trolejbusů a autobusů je statisticky významný. Na hladině významnosti 95 % zamítáme nulovou hypotézu, že škody u trolejbusů a autobusů jsou stejné. **Hypotézu potvrzujeme**: Škoda při nehodách trolejbusů je významně nižší než při nehodách autobusů. Statistický test ukázal, že tento rozdíl není náhodný, ale lze jej považovat za skutečný a významný.